Coupled RFFs for GPs notebook

In [28]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from scipy.stats import ortho_group
from scipy.stats.distributions import chi2
from tqdm import tqdm
import pandas as pd
from scipy.stats.distributions import chi2
import os
import math

In [29]:
# Hyperparameters and stuff

dataset = 'wine'
test_frac = 0.1


In [30]:
os.chdir('/path/to/dataset/dir')         #path to dataset directory

if dataset == 'abalone':
    abalone_data = pd.read_csv('abalone.data', delimiter=",")
    all_data = np.asarray(abalone_data)
    all_data = np.hstack((np.asarray([all_data[:,8]]).T, all_data[:,1:7]))
    all_data = np.asarray(all_data,dtype=float)

    #noise = 0.2
    noise = 0.8
    lengthscale = 8
    coefficient = 1

if dataset == 'wine':
    wine_data = pd.read_csv('wine.data', delimiter=",")
    all_data = np.asarray(wine_data)

    #noise = 0.24
    noise = 1
    lengthscale = 6.5
    coefficient = 1

if dataset == 'air':
    air_data = pd.read_csv('AirQualityUCI.csv', delimiter=";")
    all_data = np.asarray(air_data)[:9357,2:15]
    all_data = all_data[:400]
    for row in range(np.shape(all_data)[0]):
        for column in range(np.shape(all_data)[1]):
            all_data[row, column] = float(str(all_data[row,column]).replace(',', '.'))
    new_all_data = []
    for data in all_data:
        if data[0] != -200.0:
            new_all_data.append(data)
    all_data = np.asarray(new_all_data, dtype = float)

if dataset == 'housing':
    housing_data = pd.read_csv('HousingData.csv', delimiter=",")
    all_data = np.asarray(housing_data)
    all_data = np.hstack((np.asarray([all_data[:,13]]).T, all_data[:,:13]))
    new_all_data = []
    for data in all_data:
        if math.isnan(np.mean(data)) == False:
            new_all_data.append(data)
    all_data = np.asarray(new_all_data)

    #noise = 0.11
    noise = 1
    lengthscale = 5
    coefficient = 1.3

if dataset == 'parkinsons':
    parkinsons_data = pd.read_csv('parkinsons_updrs.data', delimiter=",")
    all_data = np.asarray(parkinsons_data)[:,5:]        #doing the 'total' task

    #noise = 0.35
    noise = 1
    lengthscale = 10
    coefficient = 0.5

if dataset == 'cpu':
    machine_data = pd.read_csv('machine.data', delimiter=",")
    all_data = np.asarray(machine_data)[:,2:]
    all_data = np.hstack((np.asarray([all_data[:,7]]).T, all_data[:,:7]))
    all_data = np.asarray(all_data, dtype = float)

    noise = 1
    lengthscale = 4
    coefficient = 10

In [31]:
#Load and preprocess data

indices = [i for i in range(len(all_data))]
np.random.shuffle(indices)
all_data = all_data[[indices]][0]

all_data = all_data[:500]

train = all_data[:int(len(all_data) * (1 - test_frac))]
test = all_data[ int(len(all_data) * (1 - test_frac)):]



y_data = train[:,0]
x_data = train[:,1:]

x_data_mean = np.mean(x_data,axis = 0)
x_data_std = np.std(x_data, axis = 0)
x_data -= x_data_mean
x_data *= (1 / x_data_std)
y_mean = np.mean(y_data)
y_data /= y_mean
x_pred = test[:,1:]
x_pred -= x_data_mean
x_pred *= (1 / x_data_std)
y_pred = test[:,0]
y_pred /= y_mean

d = np.shape(x_data)[1]


In [32]:
# choose the respective orientations of the weights vectors
m=2     #control no. samples

basis = np.eye(d)[:m]                   #orthogonal basis
#basis = np.vstack((basis,-basis))          #antiparallel directions (can include optionally)
nb_vecs = len(basis)


In [33]:
class random_features:
    
    def get_iid_rffs(xs, nb_features, lengthscale): 
    # get iid rffs
        
        for feature in range(nb_features):
            w = np.random.normal(size = (len(basis),d))
            holder = np.matmul(w,np.transpose(xs / lengthscale))
            cs_holder = np.concatenate([np.cos(holder),np.sin(holder)],axis=0)
            if feature == 0:
                rfs = cs_holder
            else:
                rfs = np.vstack((cs_holder,rfs))
        rfs = rfs.T / np.sqrt(nb_features * nb_vecs)
        return rfs

    def get_orth_rffs(xs, ws, lengthscale): 
    #get orthogonal rffs (takes frequency vector norms as input)
        
        nb_features = len(ws)
        for feature, w in enumerate(ws):
            w = np.matmul(np.diag(w),basis)
            rotation = ortho_group.rvs(d)      #get the rotation matrix to apply to all the data
            holder = np.matmul(w,np.matmul(rotation,np.transpose(xs / lengthscale)))
            cs_holder = np.concatenate([np.cos(holder),np.sin(holder)],axis=0)
            if feature == 0:
                rfs = cs_holder
            else:
                rfs = np.concatenate((np.reshape(rfs, [feature,2*nb_vecs,len(xs)]),np.reshape(cs_holder, [1,2*nb_vecs,len(xs)])),axis=0)
        rfs = np.transpose(np.reshape(rfs, [nb_features*2*nb_vecs,len(xs)])) / np.sqrt(nb_features * nb_vecs)
        return rfs
    
    def get_approx_covariance(xs, coupling, nb_features, coefficient, lengthscale, noise = None):
    #construct approx Gram matrix

        if coupling == 'iid':        
            phi = random_features.get_iid_rffs(xs, nb_features, lengthscale)
        elif coupling == 'orth':
            ws = np.sqrt(np.random.chisquare(d,size=(nb_features,len(basis))))
            phi = random_features.get_orth_rffs(xs, ws, lengthscale)
        elif coupling == 'coupled':
            w1 = np.random.random(size=(nb_features,int(len(basis)/2)))
            ws = np.hstack((w1,1-w1))
            if len(basis)%2 == 1:
                ws = np.asarray([np.hstack((ws,np.asarray([np.random.random(nb_features)]).T))])[0]
                
            ws = np.sqrt(chi2.ppf(ws, df=d))
            phi = random_features.get_orth_rffs(xs, ws, lengthscale)
        else:
            raise Exception('Coupling not recognised')

        K_approx = (phi @ phi.T) * coefficient
        if noise is not None:
            K_approx += noise**2 * np.eye(np.shape(x_data)[0])

        return K_approx
    


In [34]:
#functions for GPs stuff

def covariance(x,x_prime,lengthscale,coefficient,noise):
   #get exact covariance matrix for Gaussian kernel

   diff_mat = x[:,None,:]-x_prime[None,:,:]
   cov = coefficient * np.exp(-np.sum(diff_mat**2/(2 * lengthscale**2),axis=2))
   if noise is not None:
       cov = cov + noise ** 2 * np.eye(cov.shape[0])
   return cov

def get_gp_posterior_mean(Kdd,Kpd,y_data):
   #compute posterior mean for datapoints given kernel matrices and observed data
   return Kpd @ np.linalg.solve(Kdd, y_data)

def get_log_likelihood(K, y):
    #Takes Gram matrix and input data, returns data log marginal likelihood for corresponding DP
    n = len(y)
    L = np.linalg.cholesky(K)
    L_diag = [L[i,i] for i in range(len(L))]
    alpha = np.linalg.solve(L.T, np.linalg.solve(L,y))
    log_likelihood = -0.5 * np.dot(y,alpha) - np.sum(np.log(L_diag)) - n/2 * np.log(2 * np.pi)
    return log_likelihood


def do_regression_test(x_data, x_pred, coupling, nb_features, lengthscale, coefficient, noise, y_data, y_pred):
   #try regression and see what error is
   both_x = np.concatenate((x_data,x_pred))
   big_K = random_features.get_approx_covariance(both_x, coupling, nb_features, coefficient, lengthscale, None)
   Kdd = big_K[:len(x_data),:len(x_data)]
   if noise is not None:
      Kdd += np.eye(np.shape(x_data)[0]) * noise**2
   Kpd = big_K[len(x_data):, :len(x_data)]
   return np.mean((Kpd @ np.linalg.solve(Kdd, y_data) - y_pred)**2) / np.mean(y_pred**2)


def get_gp_posterior_mean_and_cov(Kdd, Kpd, Kpp, y_data):
   posterior_mean = Kpd @ np.linalg.solve(Kdd, y_data)
   posterior_cov = Kpp - Kpd @ np.linalg.solve(Kdd, Kpd.T)
   return posterior_mean, posterior_cov

def get_approx_Ks(x_data, x_pred, coupling, nb_features, lengthscale, coefficient, noise, y_data):
   both_x = np.concatenate((x_data,x_pred))
   big_K = random_features.get_approx_covariance(both_x, coupling, nb_features, coefficient, lengthscale, None)
   Kdd = big_K[:len(x_data),:len(x_data)]
   if noise is not None:
      Kdd += np.eye(np.shape(x_data)[0]) * noise**2
   Kpd = big_K[len(x_data):, :len(x_data)]
   Kpp = big_K[len(x_data):,len(x_data):]
   return Kdd, Kpd, Kpp

def get_KL_div(exact_mean, approx_mean, exact_cov, approx_cov):
   KL_d = np.trace(np.linalg.solve(approx_cov, exact_cov)) + (approx_mean - exact_mean).T @ np.linalg.solve(approx_cov, (approx_mean - exact_mean)) - len(exact_mean) + np.log( np.abs(np.linalg.det(approx_cov))) - np.log(np.abs(np.linalg.det(exact_cov))) 
   return KL_d
   

In [35]:
#Compute the exact Gram matrices and posterior mean

Kdd_exact = covariance(x_data, x_data, lengthscale, coefficient, noise)
Kpd_exact = covariance(x_pred, x_data, lengthscale, coefficient, None)
Kpp_exact  = covariance(x_pred, x_pred, lengthscale, coefficient, None)

exact_mean, exact_cov = get_gp_posterior_mean_and_cov(Kdd_exact, Kpd_exact, Kpp_exact, y_data)

In [36]:
couplings = ['iid','orth','coupled']
nb_features = 20

In [37]:
#for comparing log likelihoods. Not sure this is a very good metric.

if False:
    for coupling in couplings:
        nb_tests = 1000
        log = []
        for test in tqdm(range(nb_tests)):
            Kdd_approx = random_features.get_approx_covariance(x_data, coupling, nb_features, coefficient, lengthscale, noise)
            log.append(get_log_likelihood(Kdd_approx, y_data))
        print(str(np.mean(log)) + ' =- ' + str(np.std(log) / np.sqrt(nb_tests)))

In [38]:
#for comparing KL divergence between true and approx dist
if True:
    for coupling in couplings:
        nb_tests = 1000
        log1 = []
        log2 = []
        for test in tqdm(range(nb_tests)):
            Kdd_approx, Kpd_approx, Kpp_approx = get_approx_Ks(x_data, x_pred, coupling, nb_features, lengthscale, coefficient, noise, y_data)
            approx_mean, approx_cov = get_gp_posterior_mean_and_cov(Kdd_approx, Kpd_approx, Kpp_approx, y_data)
            log1.append(get_KL_div(exact_mean,approx_mean,exact_cov,approx_cov))
            log2.append(np.sum((approx_mean - exact_mean)**2) / np.sum(exact_mean**2))
        print(str(np.mean(log1)) + ' =- ' + str(np.std(log1) / np.sqrt(nb_tests)))
        print(str(np.mean(log2)) + ' =- ' + str(np.std(log2) / np.sqrt(nb_tests)))

    



100%|██████████| 1000/1000 [00:02<00:00, 497.56it/s]


3.613303048392133 =- 0.03251852031737341
0.0018638840345447344 =- 3.1529088728713635e-05


100%|██████████| 1000/1000 [00:02<00:00, 336.07it/s]


3.525878300750357 =- 0.032451201202523844
0.0017565280227200186 =- 2.7742544562127172e-05


100%|██████████| 1000/1000 [00:03<00:00, 272.84it/s]

3.443231290594891 =- 0.026905900395360963
0.0017998305621720948 =- 3.0162315071598887e-05


In [39]:
#for comparing Gram matrix approximation directly
for coupling in couplings:
    nb_tests = 1000
    log = []
    for test in tqdm(range(nb_tests)):
        log.append(np.sum((Kdd_exact - random_features.get_approx_covariance(x_data, coupling, nb_features, coefficient, lengthscale, noise))**2) / np.sum(Kdd_exact**2))
    print(str(np.mean(log)) + ' =- ' + str(np.std(log) / np.sqrt(nb_tests)))

100%|██████████| 1000/1000 [00:01<00:00, 951.78it/s]


0.004613561730956952 =- 6.426689734886874e-05


100%|██████████| 1000/1000 [00:02<00:00, 408.37it/s]


0.004250278564800378 =- 5.4904567732245176e-05


100%|██████████| 1000/1000 [00:02<00:00, 401.03it/s]

0.003950719587653793 =- 4.4856627950755134e-05


In [40]:
#for comparing predictive accuracy. Probably need to average over data splits

for coupling in couplings:
    nb_tests = 10000
    log = []
    for test in tqdm(range(nb_tests)):
        error = do_regression_test(x_data, x_pred, coupling, nb_features, lengthscale, coefficient, noise, y_data, y_pred)
        log.append(error)
    print(str(np.mean(log)) + ' =- ' + str(np.std(log) / np.sqrt(nb_tests)))

100%|██████████| 10000/10000 [00:10<00:00, 947.93it/s]


0.01552025082039408 =- 3.9699210770761034e-05


100%|██████████| 10000/10000 [00:27<00:00, 365.80it/s]


0.015369707103572886 =- 3.820772832770254e-05


100%|██████████| 10000/10000 [00:25<00:00, 388.93it/s]

0.01542612570228066 =- 3.805291658257977e-05
